In [1]:
#IMPORTING REQUIRED MODULES

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#reading required data files

newdf=pd.read_excel(r'DDW-C18-0000.xlsx') #population by bilingualism , trilinguaalism , age
c14=pd.read_excel(r'DDW-0000C-14.xls') #POPULATION IN FIVE YEAR AGE-GROUP BY RESIDENCE AND SEX

In [3]:
#renaming the columns in newdf

newnames=['statecode','districtcode','state','type','agegroup','2.persons','2.males','2.females','3.persons','3.males','3.females']
newdf.columns=newnames

In [4]:
#removing the rows that contain metadata

newdf=newdf[5:]

In [5]:
#choosing rows that have 'agegroup' not equal to total and 'type' as total

newdf=newdf[newdf['type']=='Total']
newdf=newdf[newdf['agegroup']!='Total']

In [6]:
#selecting only the required columns

newdf=newdf.iloc[:,[0,2,4,8]]

#printing the selected column names

newdf.columns

Index(['statecode', 'state', 'agegroup', '3.persons'], dtype='object')

In [7]:
#converting string to numeric values

newdf['3.persons']=pd.to_numeric(newdf['3.persons'])

In [8]:
#selecting only required columns from c14 data frame

c14=c14.iloc[:,[3,4,5]]

In [9]:
#renaming the selected columns

c14.columns=['state','age','pop']

In [10]:
#removing the rows that contain metadata

c14=c14.iloc[6:,:]

In [11]:
#converting state names in c14 to upper case

for i in range(len(c14)):
    c14.iloc[i,0]=c14.iloc[i,0].upper()

In [12]:
#removing the rows in newdf that contain age as 'age not stated'

newdf=newdf[newdf['agegroup']!='Age not stated']

In [13]:
#removing the age groups from c14 that are not present in newdf(for consistency)

c14=c14[c14['age']!='Age not stated']
c14=c14[c14['age']!='All ages']
c14=c14[c14['age']!='0-4']

In [14]:
#selecting all the rows from c14 that have age group between 30 to 49 to make them as a single group 30-49

x=c14[c14['age']=='30-34']
x=x.append(c14[c14['age']=='35-39'])
x=x.append(c14[c14['age']=='40-44'])
x=x.append(c14[c14['age']=='45-49'])

#selecting only the required columns from x

x=x.iloc[:,[0,2]]

#convertig string to numeric value

x['pop']=pd.to_numeric(x['pop'])

#grouped data frame contains state name and the total persons present in age group 30-49 in that state

grouped=x.groupby(by=['state'],as_index=False).agg(func='sum')

#adding a new column to grouped data frame 'age' as 30-49

grouped['age']='30-49'

#reordering the columns in grouped (so that it can we merged later with other dataframe)

grouped=grouped[['state', 'age', 'pop']]

#after reordering , printing current columns in grouped

print(grouped.columns)

Index(['state', 'age', 'pop'], dtype='object')


In [15]:
#selecting all the rows from c14 that have age group between 50 to 69 to make them as a single group 50-69

x1=c14[c14['age']=='50-54']
x1=x1.append(c14[c14['age']=='55-59'])
x1=x1.append(c14[c14['age']=='60-64'])
x1=x1.append(c14[c14['age']=='65-69'])

#selecting only the required columns from x1

x1=x1.iloc[:,[0,2]]

#convertig string to numeric value

x1['pop']=pd.to_numeric(x1['pop'])

#grouped1 data frame contains state name and the total persons present in age group 50-69 in that state

grouped1=x1.groupby(by=['state'],as_index=False).agg(func='sum')

#adding a new column to grouped1 data frame 'age' as 50-69

grouped1['age']='50-69'

#reordering the columns in grouped1 (so that it can we merged later with other dataframe)

grouped1=grouped1[['state', 'age', 'pop']]

#after reordering , printing current columns in grouped1

print(grouped1.columns)

Index(['state', 'age', 'pop'], dtype='object')


In [16]:
#selecting all the rows from c14 that have age as 70+ to make them as a single group 

x2=c14[c14['age']=='70-74']
x2=x2.append(c14[c14['age']=='75-79'])
x2=x2.append(c14[c14['age']=='80+'])

#selecting only the required columns from x2

x2=x2.iloc[:,[0,2]]

#convertig string to numeric value

x2['pop']=pd.to_numeric(x2['pop'])

#grouped2 data frame contains state name and the total persons present in age group 70+ in that state

grouped2=x2.groupby(by=['state'],as_index=False).agg(func='sum')

#adding a new column to grouped2 data frame 'age' as 70+

grouped2['age']='70+'

#reordering the columns in grouped2 (so that it can we merged later with other dataframe)

grouped2=grouped2[['state', 'age', 'pop']]

#after reordering , printing current columns in grouped2

print(grouped2.columns)

Index(['state', 'age', 'pop'], dtype='object')


In [17]:
#removing all the rows in c14 that contain age group 30+

c14=c14[c14['age']!='30-34']
c14=c14[c14['age']!='35-39']
c14=c14[c14['age']!='40-44']
c14=c14[c14['age']!='45-49']
c14=c14[c14['age']!='50-54']
c14=c14[c14['age']!='55-59']
c14=c14[c14['age']!='60-64']
c14=c14[c14['age']!='65-69']
c14=c14[c14['age']!='70-74']
c14=c14[c14['age']!='75-79']
c14=c14[c14['age']!='80+']

In [18]:
#appending grouped,grouped1,grouped2 data frame to c14(now the age groups from c14 and newdf are consistent)

c14=c14.append([grouped,grouped1,grouped2])

In [19]:
#getting only the statename (removing other words from statename like 'state' etc)

for i in range(len(c14)):
    if c14.iloc[i,0]=='INDIA':
        continue
    else:
        c14.iloc[i,0]=c14.iloc[i,0][8:-5]

In [20]:
#renaming the columns in c14

c14.columns=['state', 'agegroup', 'pop']

In [21]:
#merging newdf and c14 based on state name and agegroup columns(columns with same name in both dataframes)

merged=pd.merge(newdf,c14)

In [22]:
#converting string to numeric value

merged['pop']=pd.to_numeric(merged['pop'])

In [23]:
#calculating the percentage of people speaking 3 or more langs for each age group of each state

merged['percent']=merged['3.persons']/merged['pop']

In [24]:
#selecting only the required columns

merged=merged.iloc[:,[0,2,-1]]

In [25]:
#creating new dataframe 'high' to store the age group of each state that has the highest percentage of ppl speaking 3 or more
#languages

high=pd.DataFrame()

In [26]:
#adding statecode column to high data frame

g=merged.groupby(by=['statecode'],as_index=False)

#this list  will store the statecodes

statecode_list=[]

for i,j in g:
    statecode_list.append(i)
    
#assigning the statecode_list to statecode column of high dataframe

high['statecode']=statecode_list

In [27]:
#getting the highest percentage value of  age group that speaks 3 or more langs

high['percent']=merged.groupby(by=['statecode'],as_index=False).agg(func='max')['percent']

In [28]:
#merging the high and merged data frame to get the final dataframe
#the merging happens on statecode and the highest percentage value 

finalmerged=pd.merge(high,merged)

In [29]:
#reordering of columns

percent=finalmerged['percent']*100
finalmerged=finalmerged.iloc[:,[0,2]]
finalmerged['percent']=percent

In [30]:
#renaming of columns

finalmerged.columns=['state/ut', 'age-group', 'percentage']

In [31]:
#writing finalmerged dataframe into age-india.csv

finalmerged.to_csv('age-india.csv',index=False)